In [17]:
import os
os.environ['GRB_LICENSE_FILE'] = '../content/licenses/gurobi.lic'
import cobra
from cobra.io import read_sbml_model, write_sbml_model
from dnngior.gapfill_class import Gapfill
from dnngior.NN_Predictor import NN
import pandas as pd
import numpy as np
import cobra


model = read_sbml_model("../models/Draft_models/MAG013.sbml")
model

Name,MAG013
Memory address,37cace830
Number of metabolites,691
Number of reactions,583
Number of genes,367
Number of groups,342
Objective expression,1.0*bio1 - 1.0*bio1_reverse_b18f7
Compartments,"Cytosol, Extracellular"


In [53]:
from cobra import Reaction
def reverseReaction(model, reaction):
    new_reaction = Reaction(reaction.id)
    new_reaction.name = reaction.name
    new_reaction.lower_bound = reaction.lower_bound
    new_reaction.upper_bound = reaction.upper_bound

    
    new_reaction.add_metabolites({i: -1.0*reaction.metabolites[i] for i in reaction.metabolites})

    model.remove_reactions([reaction])

    
    model.add_reactions([new_reaction])
    return model

In [54]:
model2 = reverseReaction(model, model.reactions.rxn00872_c0)
model2.reactions.rxn00872_c0.build_reaction_string(use_metabolite_names=True)

'FAD [c0] + H+ [c0] + Butyryl-CoA [c0] <=> Crotonyl-CoA [c0] + FADH2 [c0]'

In [55]:
import dnngior
from cobra.io import read_sbml_model

model2.reactions.bio1.objective_coefficient = 0
model2.reactions.rxn00872_c0.objective_coefficient = 1


cobra.io.write_sbml_model(model2, "../models/edited_models/MAG013.sbml")

# Gapfilling
gapfilled_model_complete = dnngior.Gapfill(
    draftModel="../models/edited_models/MAG013.sbml",
    medium_file= None,
    objectiveName='rxn00872_c0',
)


Gap-filling database =  ModelSEED
#reactions not found in NN-keys:  40 / 583
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Flux through biomass reaction is 1.00000000
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  79385 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  39692 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  19846 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  9923 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  4961 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  2480 


Warning for adding constraints: zero or small (< 1e-13) coefficients, 

In [43]:
model.reactions.rxn00872_c0.metabolites

{<Metabolite cpd00015_c0 at 0x3432aa170>: -1.0,
 <Metabolite cpd00067_c0 at 0x3432a9900>: -1.0,
 <Metabolite cpd00120_c0 at 0x3432047c0>: -1.0,
 <Metabolite cpd00650_c0 at 0x3432072e0>: 1.0,
 <Metabolite cpd00982_c0 at 0x3432aa0e0>: 1.0}

In [ ]:
import dnngior
print("Number of reactions added:", len(gapfilled_model_complete.added_reactions))
print("~~")
for reaction in gapfilled_model_complete.added_reactions:
    print(gapfilled_model_complete.reactions.get_by_id(reaction).name)


Number of reactions added: 0
~~


In [ ]:
import os
from cobra.io import read_sbml_model


model_dir = "../../models/fixed_models"  

butyrate_ids = {"cpd00141_e0", "cpd00141_c0"}
propionate_ids = {"cpd00211_e0", "cpd00211_c0"}

# 初始化计数器
butyrate_models = 0
propionate_models = 0
total_models = 0

# 遍历所有模型文件
for file in os.listdir(model_dir):
    if file.endswith(".xml"):
        filepath = os.path.join(model_dir, file)
        try:
            model = read_sbml_model(filepath)
        
            total_models += 1

            met_ids = {met.id.lower() for met in model.metabolites}

            # 判断是否含有丁酸和丙酸代谢物
            has_butyrate = any(met in met_ids for met in butyrate_ids)
            has_propionate = any(met in met_ids for met in propionate_ids)

            if has_butyrate:
                print("but:", model.id)
                butyrate_models += 1
            if has_propionate:
                #propionate_models += 1
                print("prop:",model.id)

        except Exception as e:
            print(f"模型读取失败: {file}，错误: {e}")

# 打印统计结果
print(f"总模型数: {total_models}")
print(f"含丁酸 (cpd00141) 的模型数: {butyrate_models}")
print(f"含丙酸 (cpd00211) 的模型数: {propionate_models}")


prop: MAG228_gapfilled
prop: MAG013_gapfilled
but: MAG099_gapfilled
prop: MAG238_gapfilled
prop: MAG084_gapfilled
prop: MAG114_gapfilled
prop: MAG147_gapfilled
prop: MAG205_gapfilled
prop: MAG139_gapfilled
prop: MAG215_gapfilled
prop: MAG154_gapfilled
prop: MAG026_gapfilled
but: MAG018_gapfilled
but: MAG071_gapfilled
prop: MAG071_gapfilled
prop: MAG087_gapfilled
prop: MAG236_gapfilled
but: MAG041_gapfilled
prop: MAG174_gapfilled
prop: MAG107_gapfilled
prop: MAG169_gapfilled
prop: MAG172_gapfilled
prop: MAG230_gapfilled
prop: MAG162_gapfilled
prop: MAG101_gapfilled
prop: MAG008_gapfilled
prop: MAG039_gapfilled
prop: MAG067_gapfilled
prop: MAG210_gapfilled
prop: MAG152_gapfilled
prop: MAG025_gapfilled
prop: MAG184_gapfilled
but: MAG189_gapfilled
prop: MAG189_gapfilled
prop: MAG122_gapfilled
prop: MAG213_gapfilled
prop: MAG197_gapfilled
but: MAG010_gapfilled
prop: MAG102_gapfilled
prop: MAG240_gapfilled
prop: MAG092_gapfilled
prop: MAG059_gapfilled
prop: MAG054_gapfilled
prop: MAG233_gapf

In [ ]:
import os
import shutil
from cobra.io import read_sbml_model

model_dir = "../../models/fixed_models"

butyrate_dir = os.path.join(model_dir, "butyrate_models")
propionate_dir = os.path.join(model_dir, "propionate_models")

# 创建输出目录（如果不存在）
os.makedirs(butyrate_dir, exist_ok=True)
os.makedirs(propionate_dir, exist_ok=True)


propionate_ids = {"cpd00141_e0", "cpd00141_c0"}
butyrate_ids = {"cpd00211_e0", "cpd00211_c0"}


butyrate_models = 0
propionate_models = 0
total_models = 0

# 遍历所有模型文件
for file in os.listdir(model_dir):
    if file.endswith(".xml"):
        filepath = os.path.join(model_dir, file)

        if filepath.startswith(butyrate_dir) or filepath.startswith(propionate_dir):
            continue

        try:
            model = read_sbml_model(filepath)
            total_models += 1

            met_ids = {met.id.lower() for met in model.metabolites}

            has_butyrate = any(met in met_ids for met in butyrate_ids)
            has_propionate = any(met in met_ids for met in propionate_ids)

            if has_butyrate:
                shutil.copy(filepath, os.path.join(butyrate_dir, file))
                print("but:", model.id)
                butyrate_models += 1

            if has_propionate:
                shutil.copy(filepath, os.path.join(propionate_dir, file))
                print("prop:", model.id)
                propionate_models += 1

        except Exception as e:
            print(f"模型读取失败: {file}，错误: {e}")

# 汇总统计
print(f"\n总模型数: {total_models}")
print(f"含丁酸的模型数: {butyrate_models}")
print(f"含丙酸的模型数: {propionate_models}")


prop: MAG228_gapfilled
prop: MAG013_gapfilled
but: MAG099_gapfilled
prop: MAG238_gapfilled
prop: MAG084_gapfilled
prop: MAG114_gapfilled
prop: MAG147_gapfilled
prop: MAG205_gapfilled
prop: MAG139_gapfilled
prop: MAG215_gapfilled
prop: MAG154_gapfilled
prop: MAG026_gapfilled
but: MAG018_gapfilled
but: MAG071_gapfilled
prop: MAG071_gapfilled
prop: MAG087_gapfilled
prop: MAG236_gapfilled
but: MAG041_gapfilled
prop: MAG174_gapfilled
prop: MAG107_gapfilled
prop: MAG169_gapfilled
prop: MAG172_gapfilled
prop: MAG230_gapfilled
prop: MAG162_gapfilled
prop: MAG101_gapfilled
prop: MAG008_gapfilled
prop: MAG039_gapfilled
prop: MAG067_gapfilled
prop: MAG210_gapfilled
prop: MAG152_gapfilled
prop: MAG025_gapfilled
prop: MAG184_gapfilled
but: MAG189_gapfilled
prop: MAG189_gapfilled
prop: MAG122_gapfilled
prop: MAG213_gapfilled
prop: MAG197_gapfilled
but: MAG010_gapfilled
prop: MAG102_gapfilled
prop: MAG240_gapfilled
prop: MAG092_gapfilled
prop: MAG059_gapfilled
prop: MAG054_gapfilled
prop: MAG233_gapf